# Schematic description of how the package and the API would works

#### Sources: 

- [x] https://christophergs.com/machine%20learning/2019/03/17/how-to-deploy-machine-learning-models/
- https://christophergs.com/machine%20learning/2019/03/30/deploying-machine-learning-applications-in-shadow-mode
- https://christophergs.com/machine%20learning/2020/03/14/how-to-monitor-machine-learning-models/
- https://landing.google.com/sre/sre-book/chapters/testing-reliability/
- https://medium.com/@copyconstruct/monitoring-in-the-time-of-cloud-native-c87c7a5bfa3e
- https://cloud.google.com/blog/products/gcp/cre-life-lessons-what-is-a-dark-launch-and-what-does-it-do-for-me
- https://martinfowler.com/articles/feature-toggles.html
- [x] https://cloud.google.com/solutions/machine-learning/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning
- [x] https://cloud.google.com/blog/products/ai-machine-learning/itau-unibanco-how-we-built-a-cicd-pipeline-for-machine-learning-with-online-training-in-kubeflow
- https://research.google/pubs/pub43146/
- https://papers.nips.cc/paper/5656-hidden-technical-debt-in-machine-learning-systems.pdf
- https://developers.google.com/machine-learning/guides/rules-of-ml/?_ga=2.89822301.795599377.1589120394-1535751243.1572158536#training-serving_skew
- https://www.forbes.com/sites/forbestechcouncil/2019/04/03/why-machine-learning-models-crash-and-burn-in-production/#61e8b1f92f43
- https://developers.google.com/machine-learning/testing-debugging

#### Helpful testing tools:
- https://github.com/ionelmc/pytest-benchmark
- https://github.com/kubeflow/kubeflow
- https://netflixtechblog.com/scheduling-notebooks-348e6c14cfd6
- https://flask.palletsprojects.com/en/1.0.x/blueprints/
- https://dzone.com/articles/schema-first-api-design


Code repository - Github: https://github.com/screwdriver66/solar_prod_suvilahti

The model includes two parts: 
- A trained model package
- API that has GET and POST endpoints to work with the trained model

#### Use cases: 
A user can install the package from a resource. He might use the package in his script the following way:

>		import solar_prod_suvilahti
	
Then later in the script use: 
		
>		get_forecast()


circleci - for continuous integration end deployment

post the package to gemfury, so that people can install it properly
- but what is inside the installation? 
- and where is the model?
- what do we need heroku for?

# Testing in CI/CD pipeline

First of all, the process of CI/CD integration should not retrain the model online. It is computationally too expensive. We retrain the model offline.

Thus, if we make changes to the feature engineering pipeline, pipeline inputs and so on- we need to retrain the model. This happens offline. Thus, we need to create a set of tests that would reflect these requirements. 

Something like: 
if there are changes to the features, then circleci needs to find a .pkl file and updated version file.